# Представление гипотез

Часть гипотез можно представлять системами уравнений, например:

$e_1: f_1(x_1)=0$

$e_2: f_2(x_2)=0$

$e_3: f_3(x_3)=0$

$e_4: x_1+x_2+x_3+x_4+x_5=0$

$e_5: x_1 + 6*x_3+x_4+x_5=0$

$e_6: f_6(x_4, x_6)=0$

$e_7: f_7(x_5, x_7)=0$

<center><img src="../pictures/structures.png"></center>

## Структуры

*Опр. 1.* *Структурой* $S\left(E, V\right)$ называется пара множеств, где $E$ - 
это множество уравнений с переменными $V$, так что $|E| \leq |V| $, а так же: 1) в любом подмножестве из $k$ 
уравнений существует не менее $k$ переменных, 2) в любом подмножестве из $k$ уравнений и $r$ 
переменных $\left(k\leq r\right)$, при условии, что значения любых $r-k$ переменных выбираются случайным образом, то значения остальных $k$ переменных определяются однозначно.

Гипотеза $h$ рассматривается как набор уравнений $E_h$ и набор всех переменных $V_h$, указанных 
в $E_h$, так что пара $\left(E_h, V_h\right)$ представляет собой структуру $S\left(E_h, V_h\right)$.

Стоит отметить в опр. 1, что структуры состоят из уравнений, а переменные являются их частью только косвенно как часть уравнений. Соответственно, все операции множеств, такие как объединение, пересечение и разность, вычисляются с использованием уравнений. То есть, если $S(E, V)$ и $S(E',V')$ являются структурами, то $S' \subset S$ когда $E' \subset E$. Определяется дополнительная операция для исключения переменных, т.е. $T \coloneqq S \div S'$, чтобы обозначить структуру $T$, полученную в результате как (1) удаления уравнений $E'$ из $E$, так и 
(2) принудительного исключения переменных $V' = \cup_{f \in E'} Vars(f)$ из $E\setminus	E'$.


*Опр. 2.* Пусть есть структура $S\left(E, V\right)$, где $x_l \in V$ - переменная. $x_l$ называется *экзогенной*, если существует уравнение $f_k \in E: f_k(x_l) = 0$, т.е. $A_S(k, j) = 1$ тогда и только тогда, когда $j = l$. Иначе $x_l$ - эндогенная.

Прим. Эндогенные переменные - переменные, которые объясняются другими переменными в модели.
Экзогенные переменные - переменные, которые не объясняются другими переменными в модели.

In [10]:
# вспомогательное
from itertools import combinations

import numpy as np
from latex2sympy import strToSympy

from collections import defaultdict
import graphviz
from sympy import Symbol
import networkx as nx


def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [41]:
tex1 = r"f_1(x_1)=0"
tex4 = r"x_1+x_2+x_3+x_4+x_5=0"

class Equation:
    def __init__(self, formula) -> None:
        self.formula = formula
        self.equation = self.get_equation()
        self.vars = self.get_vars()
    
    def get_equation(self):
        equation = strToSympy(self.formula)
        return equation

    def get_vars(self):
        vars = sorted(self.equation.free_symbols, key=lambda symbol: symbol.name)
        return vars
    
e4 = Equation(formula=tex4)
e1 = Equation(formula=tex1)
e1.vars

[x_1]

*Опр. 2.* Пусть $S\left( E, V\right)$ - это структура. $S$ является *полной*, если $|E| = |V|$.

*Опр. 3.* Пусть $S\left( E, V\right)$ - это структура. Тогда $S$ является *минимальной*, 
если она является полной и нет полной структуры $S'$, такой что $S' \subset S$.

*Опр. 4.* Структурная матрица $A_S$ структуры $S\left( E, V\right)$, где $f_1, f_2, \ldots, f_n \in E;
\text{ } x_1, x_2, \ldots, x_m \in V$ - это матрица размера $|E| \times |V|$, из нулей и единиц, такая что $a_{ij} = 1$, если переменная $x_j \in Vars(f_i)$, иначе~$0$.


In [32]:
class Structure:
    def __init__(self, equations, vars=None):
        self.equations = equations
        # vars = [eq.get_vars() for eq in self.equations]
        if vars is not None:
            all_vars = set().union(*(map(lambda x: set(x.vars), self.equations)))
            subs_vars = all_vars.difference(vars)
            subs_vars_values = [(var, 0) for var in subs_vars]

            self.vars = vars
            equalities = list(map(lambda x: x.equation.subs(subs_vars_values), self.equations))
            for i in range(len(self.equations)):
                self.equations[i].equation = equalities[i]

        else:
            self.vars = set().union(*(map(lambda x: set(x.vars), self.equations)))


    def is_structure(self):
        is_structure = True
        all_subsets = powerset(self.equations)
        for subset in all_subsets:
            if subset:
                eq_num = len(subset)
                vars = set().union(*(map(lambda x: set(x.vars), subset)))
                var_num = len(vars)
                if eq_num > var_num:
                    is_structure = False
                else:
                    #TODO part b
                    pass
        return is_structure


    def is_complete(self):
        return len(self.equations) == len(self.vars)


    def is_minimal(self):
        return self.is_complete() and not self.find_minimal_structures()

    def find_minimal_structures(self):
        '''
        Returns:
        '''
        min_str = []

        if len(self.equations) == 1 and self.is_complete():
            return min_str

        all_subsets = powerset(self.equations)
        for subset in list(all_subsets)[1:-1]:
            s = Structure(equations=subset)
            left_vars = [eq.equation.free_symbols for eq in subset]
            left_vars = set(itertools.chain(*left_vars))
            s.vars = left_vars

            if s.is_complete() and not s.find_minimal_structures():
                min_str.append(s)

        return min_str

    def exogenous(self):
        exogenous = set()
        for eq in self.equations:
            if len(eq.get_vars()) == 1:
                exogenous = exogenous.union(eq.get_vars())
        return exogenous

    def endogenous(self):
        return self.vars.difference(self.exogenous())


    def build_matrix(self) -> np.matrix:

        if not self.is_complete():
            raise Exception("Structure is not complete")
        else:
            matrix = np.matrix(np.zeros((len(self.vars), len(self.vars))))
            # sorting of set is needed to preserve the order
            sorted_vars = sorted(self.vars, key=lambda x: x.name)
            for i in range(len(self.vars)):
                for j in range(len(self.equations)):
                    if sorted_vars[i] in self.equations[j].vars:
                        matrix[j, i] = 1
        return matrix
    

if __name__ == '__main__':

    tex1 = r"f_1(x_1)=0"
    tex2 = r"f_2(x_2)=0"
    tex3 = r"f_3(x_3)=0"
    tex4 = r"x_1+x_2+x_3+x_4+x_5=0"
    tex5 = r"x_1 + 6*x_3+x_4+x_5=0"
    tex6 = r"f_6(x_4, x_6)=0"
    tex7 = r"f_7(x_5, x_7)=0"

    e1 = Equation(formula=tex1)
    e2 = Equation(formula=tex2)
    e3 = Equation(formula=tex3)
    e4 = Equation(formula=tex4)
    e5 = Equation(formula=tex5)
    e6 = Equation(formula=tex6)
    e7 = Equation(formula=tex7)

    equations = [e1, e2, e3, e4, e5, e6, e7]
    s = Structure(equations)
    
    print(s.is_minimal())
    print(s.is_complete())

    print(s.build_matrix())
    print([eq.equation for m in s.find_minimal_structures() for eq in m.equations])
    print(s.exogenous(), s.endogenous())


False
True
[[1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 0. 0.]
 [1. 0. 1. 1. 1. 0. 0.]
 [0. 0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 1.]]
[Eq(f_1(x_1), 0), Eq(f_2(x_2), 0), Eq(f_3(x_3), 0)]
{x_2, x_1, x_3} {x_5, x_6, x_4, x_7}



*Опр. 5.* Пусть $S\left( E, V\right)$ - это полная структура. Тогда 
*полное причинно-следственное отображение* над $S$ - это биективное отображение $\phi: E\to V$, 
такое что $\forall f \in E$, если $\phi \left( f\right) = x$, то $x \in Vars \left( f \right)$. 
Здесь $Vars \left( f \right)$ - это множество всех переменных из уравнений $f \in E$. $|S|$ обозначает число 
появлений переменных в уравнениях, т.е. $|S\left( E, V\right)| = \sum\limits_{f \in E} Vars\left( f\right)$.


**Лемма 1.** Вычислительная сложность процедуры построения полного причинно-следственного отображения для полной структуры $S\left( E, V\right)$ ограничена $O\left(|S|*\sqrt{|V|}\right)$.

*Опр. 6.* Пусть $S\left( E, V\right)$ - это полная структура, где $x_a, x_b \in V$ и $\phi: E \to V$ 
"- это полное причинно-следственное отображение над $S$. Тогда $C_\phi = \{ \left( x_a, x_b \right) \mid 
\exists f \in E: \phi \left( f \right) = x_b, \text{ } x_a \in Vars\left( f \right) \}$ - это множество 
*прямых причинно-следственных зависимостей*. Транзитивное замыкание этого множества обозначается 
$C_\phi^+$ и является набором причинно-следственных зависимостей.  

**Лемма 2.** Вычислительная сложность процедуры построения транзитивного замыкания 
для полной структуры $S\left( E, V\right)$ ограничена $O\left(|S|*|V|\right)$. 

In [ ]:
class Structure:
    # продолжение 
    def build_transitive_closure(self):
        fcm = self.build_full_causal_mapping()
        direct_dependencies = list()
        for key, value in fcm.items():
            equation = Equation(formula=key)
            dep = equation.get_vars()
            dep.remove(value)
            for d in dep:
                direct_dependencies.append(tuple((d, value)))

        tc = self.deps_transitive_closure(direct_dependencies)
        return tc

    def deps_transitive_closure(self, direct_dependencies):
        tc = defaultdict(list)
        G = nx.DiGraph()
        for dep in direct_dependencies:
            G.add_edge(dep[0], dep[1])

        for node in G.nodes():
            tc[node] = nx.algorithms.descendants(G, node)
        return tc


    def build_full_causal_mapping(self):
        if not self.is_complete():
            raise Exception('Structure is not complete')
        else:
            left_structures = Structure(equations=self.equations, vars=self.vars)
            fcm = {}

            while(left_structures):

                if left_structures.is_minimal():
                    sorted_vars = sorted(left_structures.vars, key=lambda x: x.name)
                    for eq in left_structures.equations:
                        fcm[eq.formula] = sorted_vars[0]
                        sorted_vars = [x for x in sorted_vars if x != list(left_structures.vars)[0]]
                    break

                minimal_structures = left_structures.find_minimal_structures()

                for mstr in minimal_structures:
                    sorted_vars = sorted(mstr.vars, key=lambda x: x.name)
                    for eq in mstr.equations:
                        fcm[eq.formula] = sorted_vars[0]
                        sorted_vars = [x for x in sorted_vars if x != sorted_vars[0]]

                difference = left_structures.difference(set(minimal_structures))
                if len(difference.equations) > 0:
                    left_structures = Structure(equations=difference.equations, vars=difference.vars)
                else:
                    left_structures = None
            return fcm
    

    def union(self, set_structures):
        united = []
        for eq in self.equations:
            united.append(eq)
        for structure in set_structures:
            for eq in structure.equations:
                united.append(eq)
        return Structure(equations=united)

    def difference(self, set_structures):

        set_eq = [s.equations for s in set_structures]
        set_eq = list(itertools.chain(*set_eq))

        set_vars = [s.vars for s in set_structures]
        set_vars = list(itertools.chain(*set_vars))
        left_equations = [eq for eq in self.equations if eq not in set_eq]
        left_variables = set([v for v in self.vars if v not in set_vars])

        return Structure(equations=left_equations, vars=left_variables)


    def build_dcg(self):
        dot = graphviz.Digraph('dcg', comment='Directed Causal Graph')
        h_encode = self.h_encode()
        for key, value in h_encode.items():
            for k in key:
                for v in value:
                    dot.edge(str(k), str(v))

        return dot


    def h_encode(self):
        if not self.is_complete():
            raise Exception("Structure is not complete")
        else:
            fd = defaultdict(list)
            fcm = self.build_full_causal_mapping()
            # a_s = self.build_matrix()

            for key, value in fcm.items():
                equation = Equation(formula=key)
                dep = equation.get_vars()

                dep.remove(value)
                if not dep:
                    dep.append(Symbol('phi'))
                else:
                    dep.append(Symbol('v'))
                fd[tuple(dep)].append(value)
            return fd
        
if __name__ == '__main__':
    virtual_experiment_onto = get_ontology("http://synthesis.ipi.ac.ru/virtual_experiment.owl")

    tex1 = r"f_1(x_1)=0"
    tex2 = r"f_2(x_2)=0"
    tex3 = r"f_3(x_3)=0"
    tex4 = r"x_1+x_2+x_3+x_4+x_5=0"
    tex5 = r"x_1 + 6*x_3+x_4+x_5=0"
    tex6 = r"f_6(x_4, x_6)=0"
    tex7 = r"f_7(x_5, x_7)=0"

    e1 = Equation(formula=tex1)
    e2 = Equation(formula=tex2)
    e3 = Equation(formula=tex3)
    e4 = Equation(formula=tex4)
    e5 = Equation(formula=tex5)
    e6 = Equation(formula=tex6)
    e7 = Equation(formula=tex7)

    equations = [e1, e2, e3, e4, e5, e6, e7]
    s = Structure(equations)
    print(s.build_full_causal_mapping())
    print(s.build_matrix())
    print(s.find_minimal_structures())
    print(s.exogenous(), s.endogenous())

    print(s.h_encode())
    print(s.build_dcg().source)
    print(s.build_transitive_closure())

## Построение решетки гипотез
*Опр. 7.* Решетка гипотез $L$ представляет собой ориентированный ациклический граф, вершины которого 
соответствуют гипотезам с некоторой структурой $S$, а ребра соответствуют отношению $derived\_by$  между гипотезами. Отношение $derived\_ by\left(b, a\right)$ означает, что результат вычисления гипотезы $b$ используется при вычислении 
гипотезы $a$.

*Опр. 8.* Поток работ $W$ представляет собой ориентированный ациклический граф. Вершины $W$ называются задачами. Каждая задача рассматривается как сценарий, вызывающий некоторые функции, реализующие гипотезы. $|W|$ обозначает общее количество задач в потоке работ.

**Лемма 3.** Вычислительная сложность алгоритма построения решетки гипотез для потока работ $W$ и набора гипотез 
$H$ ограничена функцией $ O\left( |W|^2 * |S| * |V| * |H| \right)$, где 
$S = \bigcup\limits_{h \in H} S\left(E_h, V_h \right), V = \bigcup\limits_{h \in H} V_h$.